<a href="https://colab.research.google.com/github/kdubs100/CS_793/blob/main/Edited_CS_793_Assignment_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install numpy
!pip install transformers
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00


# 1st BERT model

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer

raw_data1 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample1 = np.random.choice(raw_data1['train'], 10000, replace=False)
sample1_headlines = []

for data_dict1 in sample1:
    if 'headline' in data_dict1:
        sample1_headlines.append(data_dict1['headline'])
#print(sample_headlines)


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines1 = []
batch_size = 100
i = 0
while i < len(sample1_headlines):
    batch_headlines = sample1_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines1.append(embedded)
    i += batch_size

embedded_headlines1 = np.concatenate(embedded_headlines1, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines1))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

10000


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages1 = []
for n_clusters in range_n_clusters:

    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines1)

    sample_silhouette_values1 = silhouette_samples(embedded_headlines1, cluster_labels)

    silhouette_avg1 = silhouette_score(embedded_headlines1, cluster_labels)
    averages1.append((n_clusters, silhouette_avg1))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg1,
    )

max_cluster1 = max(averages1, key=lambda x: x[1])
print(max_cluster1)

For n_clusters = 2 The average silhouette_score is : 0.05508749
For n_clusters = 3 The average silhouette_score is : 0.05076695
For n_clusters = 4 The average silhouette_score is : 0.03255307
For n_clusters = 5 The average silhouette_score is : 0.0332223
For n_clusters = 6 The average silhouette_score is : 0.031879697
For n_clusters = 7 The average silhouette_score is : 0.032359295
For n_clusters = 8 The average silhouette_score is : 0.02511706
(2, 0.05508749)


In [42]:
from sklearn.metrics.pairwise import cosine_similarity
import random

cluster_idx_to_sample = (int(max_cluster1[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines1 = [embedded_headlines1[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected1 = random.sample(cluster_headlines1, 5)
#print(selected1)

cosine_similarities1 = []
for i in range(len(selected1)):
    for j in range(len(embedded_headlines1)):
        # Calculate cosine similarity between selected headline and each embedded headline
        similarity1 = cosine_similarity([selected1[i]], [embedded_headlines1[j]])
        cosine_similarities1.append((i, j, similarity1))


one = []
two = []
three = []
four = []
five = []
for i, j, similarity in cosine_similarities1:
    if i == 0:
        k = (i, j, similarity)
        one.append(k)
    if i == 1:
        k = (i, j, similarity)
        two.append(k)
    if i == 2:
        k = (i, j, similarity)
        three.append(k)
    if i == 3:
        k = (i, j, similarity)
        four.append(k)
    if i == 4:
        k = (i, j, similarity)
        five.append(k)

one_sorted = sorted(one, key=lambda x: x[2], reverse=True)
two_sorted = sorted(two, key=lambda x: x[2], reverse=True)
three_sorted = sorted(three, key=lambda x: x[2], reverse=True)
four_sorted = sorted(four, key=lambda x: x[2], reverse=True)
five_sorted = sorted(five, key=lambda x: x[2], reverse=True)
#print(one_sorted[0])
#print(one_sorted[1])
#print(one_sorted[33])
#print(one_sorted[34])
#print(one_sorted[536])
#print(one_sorted[537])

print('Matching headlines for headline 1')
for k in range(5):
    h = one_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 2')
for k in range(5):
    h = two_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 3')
for k in range(5):
    h = three_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 4')
for k in range(5):
    h = four_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 5')
for k in range(5):
    h = five_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

Matching headlines for headline 1
headline 1 cosine similarity [[0.88108444]]
headline 2 cosine similarity [[0.87595665]]
headline 3 cosine similarity [[0.86970055]]
headline 4 cosine similarity [[0.8687748]]
headline 5 cosine similarity [[0.8686907]]
Matching headlines for headline 2
headline 1 cosine similarity [[0.86443055]]
headline 2 cosine similarity [[0.8612139]]
headline 3 cosine similarity [[0.8581019]]
headline 4 cosine similarity [[0.853295]]
headline 5 cosine similarity [[0.84416234]]
Matching headlines for headline 3
headline 1 cosine similarity [[0.9000995]]
headline 2 cosine similarity [[0.8985809]]
headline 3 cosine similarity [[0.89596015]]
headline 4 cosine similarity [[0.89491594]]
headline 5 cosine similarity [[0.89367926]]
Matching headlines for headline 4
headline 1 cosine similarity [[0.8642239]]
headline 2 cosine similarity [[0.845503]]
headline 3 cosine similarity [[0.84271705]]
headline 4 cosine similarity [[0.83917975]]
headline 5 cosine similarity [[0.836782

The cosine simialrity for each set of best headlines is relatively similar for each of the five headlines. Each stay within approximately .05 of the orginal headline. The same pattern follows for the other transformer models used, however the larger models used had even smaller changes in cosine similarities.

In [43]:
print('Matching headlines for headline 1')
l = random.randint(9,14)
print(f'Rank {l}')
h = one_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = one_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = one_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 2')
l = random.randint(9,14)
print(f'Rank {l}')
h = two_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = two_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = two_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 3')
l = random.randint(9,14)
print(f'Rank {l}')
h = three_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = three_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = three_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 4')
l = random.randint(9,14)
print(f'Rank {l}')
h = four_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = four_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = four_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 5')
l = random.randint(9,14)
print(f'Rank {l}')
h = five_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = five_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = five_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

Matching headlines for headline 1
Rank 11
headline 1 cosine similarity [[0.86161256]]
Rank 15
headline 2 cosine similarity [[0.85230416]]
Rank 214
headline 3 cosine similarity [[0.8113201]]

Matching headlines for headline 2
Rank 9
headline 1 cosine similarity [[0.84067476]]
Rank 20
headline 2 cosine similarity [[0.83202434]]
Rank 159
headline 3 cosine similarity [[0.8100978]]

Matching headlines for headline 3
Rank 9
headline 1 cosine similarity [[0.8873952]]
Rank 16
headline 2 cosine similarity [[0.88048184]]
Rank 172
headline 3 cosine similarity [[0.833146]]

Matching headlines for headline 4
Rank 9
headline 1 cosine similarity [[0.82578456]]
Rank 20
headline 2 cosine similarity [[0.8189608]]
Rank 219
headline 3 cosine similarity [[0.7832811]]

Matching headlines for headline 5
Rank 13
headline 1 cosine similarity [[0.81389296]]
Rank 17
headline 2 cosine similarity [[0.810127]]
Rank 427
headline 3 cosine similarity [[0.74510133]]


The results of the cosine similarities indicate that there is not a large drop off in cosine value as the value changes further away from the orginal headline. This same pattern follows for the other models, but like mentioned above a smaller change in the cosine similairites as the headlines move further from the orginal.

# 2nd BERT model


In [44]:
raw_data2 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample2 = np.random.choice(raw_data2['train'], 10000, replace=False)
sample2_headlines = []

for data_dict2 in sample2:
    if 'headline' in data_dict2:
        sample2_headlines.append(data_dict2['headline'])
#print(sample_headlines)


model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines2 = []
batch_size = 100
i = 0
while i < len(sample2_headlines):
    batch_headlines = sample2_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines2.append(embedded)
    i += batch_size

embedded_headlines2 = np.concatenate(embedded_headlines2, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines2))

10000


In [45]:
range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages2 = []
for n_clusters in range_n_clusters:

    # Initialize the clusterer with n_clusters value and other parameters as needed
    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines2)

    # Compute the silhouette scores for each sample
    sample_silhouette_values2 = silhouette_samples(embedded_headlines2, cluster_labels)

    # Calculate the average silhouette score for the clustering
    silhouette_avg2 = silhouette_score(embedded_headlines2, cluster_labels)
    averages2.append((n_clusters, silhouette_avg2))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg2,
    )

max_cluster2 = max(averages2, key=lambda x: x[1])
print(max_cluster2)

For n_clusters = 2 The average silhouette_score is : 0.30789903
For n_clusters = 3 The average silhouette_score is : 0.099289335
For n_clusters = 4 The average silhouette_score is : 0.055882704
For n_clusters = 5 The average silhouette_score is : 0.038564455
For n_clusters = 6 The average silhouette_score is : 0.036352858
For n_clusters = 7 The average silhouette_score is : 0.033684004
For n_clusters = 8 The average silhouette_score is : 0.029727835
(2, 0.30789903)


In [46]:
from sklearn.metrics.pairwise import cosine_similarity
import random

cluster_idx_to_sample = (int(max_cluster1[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines2 = [embedded_headlines2[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected2 = random.sample(cluster_headlines2, 5)
#print(selected1)

cosine_similarities2 = []
for i in range(len(selected2)):
    for j in range(len(embedded_headlines2)):
        # Calculate cosine similarity between selected headline and each embedded headline
        similarity2 = cosine_similarity([selected2[i]], [embedded_headlines2[j]])
        cosine_similarities2.append((i, j, similarity2))


one2 = []
two2 = []
three2 = []
four2 = []
five2 = []
for i, j, similarity in cosine_similarities2:
    if i == 0:
        k = (i, j, similarity)
        one2.append(k)
    if i == 1:
        k = (i, j, similarity)
        two2.append(k)
    if i == 2:
        k = (i, j, similarity)
        three2.append(k)
    if i == 3:
        k = (i, j, similarity)
        four2.append(k)
    if i == 4:
        k = (i, j, similarity)
        five2.append(k)

one2_sorted = sorted(one2, key=lambda x: x[2], reverse=True)
two2_sorted = sorted(two2, key=lambda x: x[2], reverse=True)
three2_sorted = sorted(three2, key=lambda x: x[2], reverse=True)
four2_sorted = sorted(four2, key=lambda x: x[2], reverse=True)
five2_sorted = sorted(five2, key=lambda x: x[2], reverse=True)
#print(one_sorted[0])
#print(one_sorted[1])
#print(one_sorted[33])
#print(one_sorted[34])
#print(one_sorted[536])
#print(one_sorted[537])

print('Matching headlines for headline 1')
for k in range(5):
    h = one2_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 2')
for k in range(5):
    h = two2_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 3')
for k in range(5):
    h = three2_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 4')
for k in range(5):
    h = four2_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 5')
for k in range(5):
    h = five2_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')



Matching headlines for headline 1
headline 1 cosine similarity [[0.9466862]]
headline 2 cosine similarity [[0.94472027]]
headline 3 cosine similarity [[0.944457]]
headline 4 cosine similarity [[0.9423919]]
headline 5 cosine similarity [[0.9418676]]
Matching headlines for headline 2
headline 1 cosine similarity [[0.948093]]
headline 2 cosine similarity [[0.9452356]]
headline 3 cosine similarity [[0.9446863]]
headline 4 cosine similarity [[0.9438895]]
headline 5 cosine similarity [[0.9434058]]
Matching headlines for headline 3
headline 1 cosine similarity [[0.9515138]]
headline 2 cosine similarity [[0.9503424]]
headline 3 cosine similarity [[0.946826]]
headline 4 cosine similarity [[0.94643366]]
headline 5 cosine similarity [[0.94508827]]
Matching headlines for headline 4
headline 1 cosine similarity [[0.9474826]]
headline 2 cosine similarity [[0.94464916]]
headline 3 cosine similarity [[0.94458747]]
headline 4 cosine similarity [[0.9433228]]
headline 5 cosine similarity [[0.9420755]]
Ma

In [47]:
print('Matching headlines for headline 1')
l = random.randint(9,14)
print(f'Rank {l}')
h = one2_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = one2_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = one2_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 2')
l = random.randint(9,14)
print(f'Rank {l}')
h = two2_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = two2_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = two2_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 3')
l = random.randint(9,14)
print(f'Rank {l}')
h = three2_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = three2_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = three2_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 4')
l = random.randint(9,14)
print(f'Rank {l}')
h = four2_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = four2_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = four2_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 5')
l = random.randint(9,14)
print(f'Rank {l}')
h = five2_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = five2_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = five2_sorted[l][2]
print(f'headline {3} cosine similarity {h}')


Matching headlines for headline 1
Rank 13
headline 1 cosine similarity [[0.93937427]]
Rank 15
headline 2 cosine similarity [[0.9387895]]
Rank 443
headline 3 cosine similarity [[0.9217917]]

Matching headlines for headline 2
Rank 14
headline 1 cosine similarity [[0.93960667]]
Rank 19
headline 2 cosine similarity [[0.93889076]]
Rank 824
headline 3 cosine similarity [[0.91749513]]

Matching headlines for headline 3
Rank 12
headline 1 cosine similarity [[0.9408883]]
Rank 19
headline 2 cosine similarity [[0.9396336]]
Rank 414
headline 3 cosine similarity [[0.9218786]]

Matching headlines for headline 4
Rank 12
headline 1 cosine similarity [[0.9359586]]
Rank 18
headline 2 cosine similarity [[0.9345467]]
Rank 841
headline 3 cosine similarity [[0.91120803]]

Matching headlines for headline 5
Rank 14
headline 1 cosine similarity [[0.93868595]]
Rank 15
headline 2 cosine similarity [[0.9384647]]
Rank 700
headline 3 cosine similarity [[0.9178666]]


# 3rd BERT Model

In [48]:
raw_data3 = load_dataset(
    'dell-research-harvard/headlines-semantic-similarity',
    data_files=["1929_headlines.json"]
)
#print(raw_data)

sample3 = np.random.choice(raw_data3['train'], 10000, replace=False)
sample3_headlines = []

for data_dict3 in sample3:
    if 'headline' in data_dict3:
        sample3_headlines.append(data_dict3['headline'])
#print(sample_headlines)


model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#embeddings created in batches
embedded_headlines3 = []
batch_size = 100
i = 0
while i < len(sample3_headlines):
    batch_headlines = sample3_headlines[i:i+batch_size]
    max_seq_length = 16
    input_headlines = tokenizer(batch_headlines, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_length)
    outputs_headlines = model(**input_headlines)
    embedded = outputs_headlines.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    embedded_headlines3.append(embedded)
    i += batch_size

embedded_headlines3 = np.concatenate(embedded_headlines3, axis=0)

#print(embedded_headlines)
print(len(embedded_headlines3))

10000


In [49]:
range_n_clusters = [ 2, 3, 4, 5, 6, 7, 8]
averages3 = []
for n_clusters in range_n_clusters:

    # Initialize the clusterer with n_clusters value and other parameters as needed
    clusterer = KMeans(n_clusters=n_clusters, init='random', n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(embedded_headlines2)

    # Compute the silhouette scores for each sample
    sample_silhouette_values3 = silhouette_samples(embedded_headlines3, cluster_labels)

    # Calculate the average silhouette score for the clustering
    silhouette_avg3 = silhouette_score(embedded_headlines3, cluster_labels)
    averages3.append((n_clusters, silhouette_avg2))
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg3,
    )

max_cluster3 = max(averages3, key=lambda x: x[1])
print(max_cluster3)

For n_clusters = 2 The average silhouette_score is : 0.0005813392
For n_clusters = 3 The average silhouette_score is : -0.0018982353
For n_clusters = 4 The average silhouette_score is : -0.0021773255
For n_clusters = 5 The average silhouette_score is : -0.004838877
For n_clusters = 6 The average silhouette_score is : -0.004623622
For n_clusters = 7 The average silhouette_score is : -0.0059550446
For n_clusters = 8 The average silhouette_score is : -0.006055265
(2, 0.029727835)


In [50]:
from sklearn.metrics.pairwise import cosine_similarity
import random

cluster_idx_to_sample = (int(max_cluster1[0])-1)
clusterer = KMeans(n_clusters=cluster_idx_to_sample, init='random', n_init="auto", random_state=10)
cluster_idx_to_print = cluster_idx_to_sample


cluster_headlines2 = [embedded_headlines2[i] for i, label in enumerate(cluster_labels) if label == cluster_idx_to_print]
#print(f"Cluster {cluster_idx_to_print + 1} Headlines:")
#for i, headline in enumerate(cluster_headlines1):
#    print(f"Headline {i + 1}: {headline}")

selected2 = random.sample(cluster_headlines2, 5)
#print(selected1)

cosine_similarities2 = []
for i in range(len(selected2)):
    for j in range(len(embedded_headlines2)):
        # Calculate cosine similarity between selected headline and each embedded headline
        similarity2 = cosine_similarity([selected2[i]], [embedded_headlines2[j]])
        cosine_similarities2.append((i, j, similarity2))


one3 = []
two3 = []
three3 = []
four3 = []
five3 = []
for i, j, similarity in cosine_similarities2:
    if i == 0:
        k = (i, j, similarity)
        one3.append(k)
    if i == 1:
        k = (i, j, similarity)
        two3.append(k)
    if i == 2:
        k = (i, j, similarity)
        three3.append(k)
    if i == 3:
        k = (i, j, similarity)
        four3.append(k)
    if i == 4:
        k = (i, j, similarity)
        five3.append(k)

one3_sorted = sorted(one2, key=lambda x: x[2], reverse=True)
two3_sorted = sorted(two2, key=lambda x: x[2], reverse=True)
three3_sorted = sorted(three2, key=lambda x: x[2], reverse=True)
four3_sorted = sorted(four2, key=lambda x: x[2], reverse=True)
five3_sorted = sorted(five2, key=lambda x: x[2], reverse=True)
#print(one_sorted[0])
#print(one_sorted[1])
#print(one_sorted[33])
#print(one_sorted[34])
#print(one_sorted[536])
#print(one_sorted[537])

print('Matching headlines for headline 1')
for k in range(5):
    h = one3_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 2')
for k in range(5):
    h = two3_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 3')
for k in range(5):
    h = three3_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 4')
for k in range(5):
    h = four3_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')

print('Matching headlines for headline 5')
for k in range(5):
    h = five3_sorted[k+1][2]
    print(f'headline {k+1} cosine similarity {h}')



Matching headlines for headline 1
headline 1 cosine similarity [[0.9466862]]
headline 2 cosine similarity [[0.94472027]]
headline 3 cosine similarity [[0.944457]]
headline 4 cosine similarity [[0.9423919]]
headline 5 cosine similarity [[0.9418676]]
Matching headlines for headline 2
headline 1 cosine similarity [[0.948093]]
headline 2 cosine similarity [[0.9452356]]
headline 3 cosine similarity [[0.9446863]]
headline 4 cosine similarity [[0.9438895]]
headline 5 cosine similarity [[0.9434058]]
Matching headlines for headline 3
headline 1 cosine similarity [[0.9515138]]
headline 2 cosine similarity [[0.9503424]]
headline 3 cosine similarity [[0.946826]]
headline 4 cosine similarity [[0.94643366]]
headline 5 cosine similarity [[0.94508827]]
Matching headlines for headline 4
headline 1 cosine similarity [[0.9474826]]
headline 2 cosine similarity [[0.94464916]]
headline 3 cosine similarity [[0.94458747]]
headline 4 cosine similarity [[0.9433228]]
headline 5 cosine similarity [[0.9420755]]
Ma

In [ ]:
print('Matching headlines for headline 1')
l = random.randint(9,14)
print(f'Rank {l}')
h = one3_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = one3_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = one3_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 2')
l = random.randint(9,14)
print(f'Rank {l}')
h = two3_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = two3_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = two3_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 3')
l = random.randint(9,14)
print(f'Rank {l}')
h = three3_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = three3_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = three3_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 4')
l = random.randint(9,14)
print(f'Rank {l}')
h = four3_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = four3_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = four3_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

print('\nMatching headlines for headline 5')
l = random.randint(9,14)
print(f'Rank {l}')
h = five3_sorted[l][2]
print(f'headline {1} cosine similarity {h}')
l = random.randint(15,20)
print(f'Rank {l}')
h = five3_sorted[l][2]
print(f'headline {2} cosine similarity {h}')
l = random.randint(100,999)
print(f'Rank {l}')
h = five3_sorted[l][2]
print(f'headline {3} cosine similarity {h}')

